In [ ]:
!nvidia-smi

Fri Aug 13 13:38:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/KaggleData/coleridgeinitiative-show-us-the-data.zip

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 7.4 MB/s 
     |████████████████████████████████| 3.3 MB 26.2 MB/s 
     |████████████████████████████████| 636 kB 19.2 MB/s 
     |████████████████████████████████| 895 kB 54.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd 
import numpy as np 
import json 
import seaborn as sns 
import math 
from matplotlib import pyplot as plt 
from tokenizers import ByteLevelBPETokenizer
import torch 
from torch import nn
from torch.utils.data import Dataset,DataLoader,TensorDataset
from transformers import RobertaModel,RobertaTokenizer,GPT2Model,GPT2TokenizerFast,AdamW,get_linear_schedule_with_warmup,RobertaConfig,GPT2Config
import os 
import random 
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm 
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
tqdm.pandas()
import gc 
from tokenizers import ByteLevelBPETokenizer
gc.enable()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
os.environ['KAGGLE_USERNAME']='daominhkhanh'
os.environ['KAGGLE_KEY']="a6b5596f1c2c9adbeda916b0ec27faba"
!kaggle datasets download -d daominhkhanh/gpt2tokenizer
!kaggle datasets download -d daominhkhanh/robertabasetokenizer

  0% 0.00/556k [00:00<?, ?B/s]
100% 556k/556k [00:00<00:00, 75.9MB/s]
  0% 0.00/541k [00:00<?, ?B/s]
100% 541k/541k [00:00<00:00, 80.2MB/s]


In [ ]:
!unzip gpt2tokenizer.zip 
!mkdir GPT2 
!mv *.json GPT2
!mv merges.txt GPT2

!unzip robertabasetokenizer.zip
!mkdir Roberta
!mv *.json Roberta
!mv merges.txt Roberta

Archive:  gpt2tokenizer.zip
  inflating: config.json             
  inflating: merges.txt              
  inflating: vocab.json              
Archive:  robertabasetokenizer.zip
  inflating: config.json             
  inflating: merges.txt              
  inflating: vocab.json              


In [ ]:
!rm *.zip

In [ ]:
train=pd.read_csv('train.csv')
sample_submission=pd.read_csv("sample_submission.csv")

In [ ]:
print(f"We have {len(os.listdir('/content/train'))} file json for train")
print(f"we have {len(os.listdir('/content/test'))} file json for test")

We have 14316 file json for train
we have 4 file json for test


In [ ]:
import re
def clean_text(text):
  return re.sub('[^A-Za-z0-9]+',' ',text.lower()).strip()

In [ ]:
class Config:
  MAX_LENGTH=256
  WINDOWN_SIZE=28
  IS_LOWER=False
  

In [ ]:
tokenizer=ByteLevelBPETokenizer(
    vocab='/content/Roberta/vocab.json',
    merges='/content/Roberta/merges.txt',
    add_prefix_space=False,
    lowercase=False
)

In [ ]:
train_df=pd.DataFrame()
texts=[]
pub_titles=[]
labels=[]
ids=[]
for idx,row in tqdm(train.iterrows(),total=len(train)):
  data_json=json.load(open(f'/content/train/{row.Id}.json','rt'))
  text=""
  for section in data_json:
    text+=" ".join(section['text'].replace('\n',' ').split())+" "
  text=clean_text(text)
  input_ids=tokenizer.encode(text,add_special_tokens=False).ids
  n_sample=math.ceil(len(input_ids)/(Config.MAX_LENGTH-Config.WINDOWN_SIZE))
  label=clean_text(row.dataset_label)
  if label not in text:
    print(row.Id)
  for sample in range(n_sample):
    start=(Config.MAX_LENGTH-Config.WINDOWN_SIZE)*sample
    end=min(start+Config.MAX_LENGTH,len(input_ids))
    curr_ids=input_ids[start:end]
    curr_text=tokenizer.decode(curr_ids)
    texts.append(curr_text)
    if label in curr_text:
      labels.append(label)
    else:
      labels.append("")
    pub_titles.append(row.pub_title)
    ids.append(row.Id)

In [ ]:
train_df['id']=ids
train_df['text']=texts
train_df['pub_titles']=pub_titles
train_df['label']=labels
train_df=train_df.fillna("")
train_df.to_csv(f"/content/drive/MyDrive/ColeridgeInitiative/train_processed_{Config.MAX_LENGTH}.csv")

In [ ]:
train_df

,id,text,pub_titles,label
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,this study used data from the national educati...,The Impact of Dual Enrollment on College Degre...,national education longitudinal study
1,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,coursework and enabling students to accumulat...,The Impact of Dual Enrollment on College Degre...,
2,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,enrollment course were not more likely to att...,The Impact of Dual Enrollment on College Degre...,
3,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,the nels sample who attended college and part...,The Impact of Dual Enrollment on College Degre...,national education longitudinal study
4,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,were included in the study the effect size is...,The Impact of Dual Enrollment on College Degre...,
...,...,...,...,...
855378,fd23e7e0-a5d2-4f98-992d-9209c85153bb,db 4 unique structures have been fetched from ...,A ligand-based computational drug repurposing ...,
855379,fd23e7e0-a5d2-4f98-992d-9209c85153bb,structural query it appears interesting that ...,A ligand-based computational drug repurposing ...,
855380,fd23e7e0-a5d2-4f98-992d-9209c85153bb,steps of the workflow tutorials and the diffe...,A ligand-based computational drug repurposing ...,
855381,fd23e7e0-a5d2-4f98-992d-9209c85153bb,silico drug repurposing the consecutive data ...,A ligand-based computational drug repurposing ...,


In [ ]:
# # Finding missing abbreviation of training labels

acronyms = {
    "alzheimer's disease neuroimaging initiative": "adni",
    "alzheimer's disease cooperative study": "adcs",
    "baltimore longitudinal study of aging":  "blsa",
    "covid-19 open research dataset": "cord-19",
#     "census of agriculture": "coa",
    "characterizing health associated risks and your baseline disease in sars cov 2": "charybdis",
    "nces core of common data": "ccd",
    "complexity science hub covid 19 control strategies list": "cccsl",
    "coastal change analysis program": "c-cap",
    "noaa national water level observation network": "nwlon",
    "noaa sea lake and overland surges from hurricanes": "slosh",
    "international best-track archive for climate stewardship": "ibtracs",
    "optimum interpolation sea surface temperature": "oisst",
    "rural-urban continuum codes": "ruccs",
    "north american breeding bird survey": "bbs",
    "aging integrated database": "agid",
    "national institute on aging genetics of alzheimer s disease data storage site": "niagads",
#     "agricultural and resource management survey": "arm",
    "baccalaureate and beyond longitudinal study": "b&b",
    "early childhood longitudinal study": "ecls",
    "national longitudinal transition study": "nlts",
    "national education longitudinal studies": "nels",
    "high school longitudinal study": "hsls",
    "national assessment of education progress": "naep",
    "noaa world ocean database": "wod",
    "survey of doctorate recipients": "sdr",
    "international survey of doctoral recipients": "isdr",
#     "survey of earned doctorates": "sed",
    "survey of industrial research and development": "sird",
    "national teacher and principal survey": "ntps",
    "international assessment of adult competencies": "piaac",
    "rsna international covid 19 open radiology database": "ricord",
    "survey of doctorate recipients": "sdr",
    "school survey on crime and safety": "ssocs",
    "trends in international mathematics and science survey": "timss",
    "national postsecondary student aid survey": "npsas",
#     "postsecondary longitudinal studies" : "pls"
}
for key, val in acronyms.items():
    acronyms[key] = val.upper()

In [ ]:
acronyms.values()

dict_values(['ADNI', 'ADCS', 'BLSA', 'CORD-19', 'CHARYBDIS', 'CCD', 'CCCSL', 'C-CAP', 'NWLON', 'SLOSH', 'IBTRACS', 'OISST', 'RUCCS', 'BBS', 'AGID', 'NIAGADS', 'B&B', 'ECLS', 'NLTS', 'NELS', 'HSLS', 'NAEP', 'WOD', 'SDR', 'ISDR', 'SIRD', 'NTPS', 'PIAAC', 'RICORD', 'SSOCS', 'TIMSS', 'NPSAS'])

In [ ]:
val_group=[clean_text(x) for x in open('val_groups.txt','rt').readlines()]
train_group=[clean_text(x) for x in set(train_df.label.values) if clean_text(x) not in val_group and len(x)>0]
train_group.extend([clean_text(x) for x in acronyms.values()])
val_group=np.array(val_group)
train_group=np.array(train_group)

In [ ]:
def find_validation(row):
  text=row.text
  is_train,is_val=False,False
  for label in train_group:
    if label in text:
      if len(label.split())>1 or " "+label in text:
        is_train=True
        break
  for label in val_group:
    if label in text:
      is_val=True
      break
  if is_train is True and is_val is True:
    return None 
  if is_train is True and is_val is False:
    return 'train'
  if is_train is False and is_val is True:
    return 'val'
  if is_train is False and is_val is False:
    return 'train' if np.random.rand()>0.2 else 'val'
train_df['group']=train_df.progress_apply(find_validation,axis=1)

In [ ]:
val_df=train_df[train_df.group=='val']
train_df=train_df[train_df.group=='train']

In [ ]:
def find_val_label(x):
  labels=[]
  for label in val_group:
    if label in x:
      labels.append(label)
  return '|'.join(labels)
val_df.label=val_df.text.progress_apply(find_val_label)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
train_df.to_csv(f'/content/drive/MyDrive/ColeridgeInitiative/train_group_{Config.MAX_LENGTH}.csv',index=False)
val_df.to_csv(f'/content/drive/MyDrive/ColeridgeInitiative/val_group_{Config.MAX_LENGTH}.csv',index=False)

In [ ]:
train_df

,id,text,pub_titles,label,group
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,this study used data from the national educati...,The Impact of Dual Enrollment on College Degre...,national education longitudinal study,train
1,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,coursework and enabling students to accumulat...,The Impact of Dual Enrollment on College Degre...,,train
2,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,enrollment course were not more likely to att...,The Impact of Dual Enrollment on College Degre...,,train
3,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,the nels sample who attended college and part...,The Impact of Dual Enrollment on College Degre...,national education longitudinal study,train
4,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,were included in the study the effect size is...,The Impact of Dual Enrollment on College Degre...,,train
...,...,...,...,...,...
855376,fd23e7e0-a5d2-4f98-992d-9209c85153bb,section in total 257 distinct mcss were calcu...,A ligand-based computational drug repurposing ...,,train
855378,fd23e7e0-a5d2-4f98-992d-9209c85153bb,db 4 unique structures have been fetched from ...,A ligand-based computational drug repurposing ...,,train
855379,fd23e7e0-a5d2-4f98-992d-9209c85153bb,structural query it appears interesting that ...,A ligand-based computational drug repurposing ...,,train
855380,fd23e7e0-a5d2-4f98-992d-9209c85153bb,steps of the workflow tutorials and the diffe...,A ligand-based computational drug repurposing ...,,train


In [ ]:
#select all the positive samples and 5% of all negative samples
train_df['will_take']=train_df.label.apply(lambda x: len(x)>0 or np.random.rand()<0.05)
train_df[train_df.will_take][train_df.columns[:-1]].to_csv(f'/content/drive/MyDrive/ColeridgeInitiative/train_sampled_{Config.MAX_LENGTH}.csv',index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
train_sampled=pd.read_csv(f'/content/drive/MyDrive/ColeridgeInitiative/train_sampled_{Config.MAX_LENGTH}.csv')

In [ ]:
train_sampled=train_sampled.fillna("")

In [ ]:
train_sampled

,id,text,pub_titles,label,group
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,this study used data from the national educati...,The Impact of Dual Enrollment on College Degre...,national education longitudinal study,train
1,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,enrollment course were not more likely to att...,The Impact of Dual Enrollment on College Degre...,,train
2,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,the nels sample who attended college and part...,The Impact of Dual Enrollment on College Degre...,national education longitudinal study,train
3,2f26f645-3dec-485d-b68d-f013c9e05e60,dropping out of high school is not necessarily...,Educational Attainment of High School Dropouts...,national education longitudinal study,train
4,2f26f645-3dec-485d-b68d-f013c9e05e60,as dropouts after they had dropped out of hig...,Educational Attainment of High School Dropouts...,national education longitudinal study,train
...,...,...,...,...,...
74995,fd23e7e0-a5d2-4f98-992d-9209c85153bb,have been collected for protein x and these a...,A ligand-based computational drug repurposing ...,,train
74996,fd23e7e0-a5d2-4f98-992d-9209c85153bb,silico drug repurposing the consecutive data ...,A ligand-based computational drug repurposing ...,,train
74997,fd23e7e0-a5d2-4f98-992d-9209c85153bb,target identification and prioritization by p...,A ligand-based computational drug repurposing ...,cas covid 19 antiviral candidate compounds data,train
74998,fd23e7e0-a5d2-4f98-992d-9209c85153bb,bank 7 and the cas covid 19 antiviral candida...,A ligand-based computational drug repurposing ...,cas covid 19 antiviral candidate compounds data,train


In [ ]:
train_sampled['length']=train_sampled.label.apply(lambda x:len(x))
train_sampled.describe()

,length
count,75000.000000
mean,12.598627
std,17.200965
min,0.000000
25%,0.000000
50%,4.000000
75%,27.000000
max,128.000000


In [ ]:
print(len(train_sampled[train_sampled.length==0]))

31505


In [ ]:
train_sampled.text=train_sampled.text.apply(lambda x:" ".join(x.lower().split(" ")))
train_sampled.label=train_sampled.label.apply(lambda x:" ".join(x.lower().split(" ")))

In [ ]:
for idx,row in tqdm(train_sampled.iterrows(),total=len(train_sampled)):
  text=row.text
  label=row.label
  if text.find(label)==-1:
    print(row.id)

In [ ]:
def get_index(text,label,offsets):
  start_index=text.find(label)
  if start_index!=-1:
    end_index=start_index+len(label)-1
    temp=[0]*len(text)
    for i in range(start_index,end_index+1):
      temp[i]=1
    list_index=[]
    for i,(offset1,offset2) in enumerate(offsets):
      if sum(temp[offset1:offset2]) >0:
        list_index.append(i)
    return list_index[0],list_index[-1]
  return None,None

In [ ]:
vocab=tokenizer.get_vocab()
pad_token_idx = vocab['<pad>']
cls_token_idx = vocab['<s>']
sep_token_idx = vocab['</s>']

In [ ]:
def convert_lines(tokenizer,df,max_sequence_length=512,is_test=False):
  count_temp=0
  outputs=np.zeros((len(df),max_sequence_length+2))
  type_outputs=np.zeros((len(df),max_sequence_length+2))
  position_outputs=np.zeros((len(df),2))
  offsets_outputs=np.ones((len(df),))
  for idx,row in tqdm(df.iterrows(),total=len(df)):
    token=tokenizer.encode(row.text,add_special_tokens=False)
    input_ids=[cls_token_idx]+token.ids+[sep_token_idx]
    offsets=[(0,0)]+token.offsets+[(0,0)]
    if len(input_ids)>max_sequence_length+2:
      input_ids=input_ids[:max_sequence_length]
      input_ids[-1]=sep_token_idx
    else:
      input_ids=input_ids+[pad_token_idx]*(max_sequence_length+2-len(input_ids))
    token_type_id=[0]*len(input_ids)
    outputs[idx,:]=np.array(input_ids)
    type_outputs[idx,:]=np.array(token_type_id)
    if is_test:
      continue
    label=row.label
    if len(label)==0:
      position_outputs[idx,:]=[0,0]
    else:
      start_idx,end_idx=get_index(row.text,label,offsets)
      if start_idx is None:
        start_idx,end_idx=0,0
        count_temp+=1
      position_outputs[idx,:]=[start_idx,end_idx]
  print(f"Number sentence:{count_temp}")
  if is_test:
    return outputs,type_outputs
  else:
    return outputs,type_outputs,position_outputs

In [ ]:
outputs,type_outputs,potision_outputs=convert_lines(tokenizer,train_sampled,Config.MAX_LENGTH)


Number sentence:0


In [ ]:
print(max(potision_outputs[:,1]))

256.0


In [ ]:
import pickle 
pickle.dump((outputs,type_outputs,potision_outputs),open(f"/content/drive/MyDrive/ColeridgeInitiative/train.pickle",'wb'),)